In [2]:
import pandas
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
import pickle
from google.cloud import storage

# Authentication
import os
#os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "path_to_your_service_account_key.json"

# Set up GCS client
client = storage.Client()
bucket_name = "mybucket_004788"
bucket = client.get_bucket(bucket_name)

url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
dataframe = pandas.read_csv(url, names=names)
array = dataframe.values
X = array[:,0:8]
Y = array[:,8]
test_size = 0.33
seed = 7
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, test_size=test_size, random_state=seed)

# Fit the model on training set
model = LogisticRegression(solver='lbfgs', max_iter=1000)
model.fit(X_train, Y_train)

# Save the model to GCS bucket
filename = 'finalized_model.sav'
blob = bucket.blob(filename)
blob.upload_from_string(pickle.dumps(model))

# some time later...
# Load the model from GCS bucket
blob = bucket.get_blob(filename)

# Download model as binary
model_binary = blob.download_as_bytes()

# Deserialize the model from the binary
loaded_model = pickle.loads(model_binary)

result = loaded_model.score(X_test, Y_test)
print(result)



0.7874015748031497
